<a href="https://colab.research.google.com/github/radmm/MoSe2-structure-viewer/blob/main/MoSe2_viewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoSe₂ 3D Structure & Band Structure Viewer
**Cell 1** — displays the interactive viewer inline in Colab  
**Cell 2** — downloads the HTML file to open in Safari on iPad

In [1]:
from IPython.display import HTML, display

html_code = '''
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no">
<title>MoSe2 Viewer</title>
<style>
  @import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');
  :root { --mo:#FF6B35; --se:#A78BFA; --bg:#050a14; --panel:rgba(10,20,40,0.85); --accent:#00e5ff; --text:#cdd9e5; }
  * { margin:0; padding:0; box-sizing:border-box; }
  body { background:var(--bg); color:var(--text); font-family:'Inter',sans-serif; overflow:hidden; height:600px; width:100%; }
  #app { display:flex; flex-direction:column; height:600px; }
  header { padding:12px 20px; background:var(--panel); border-bottom:1px solid rgba(0,229,255,0.2); display:flex; align-items:center; justify-content:space-between; flex-shrink:0; backdrop-filter:blur(12px); z-index:10; }
  header h1 { font-size:1.3rem; font-weight:800; letter-spacing:-0.02em; color:#fff; }
  header h1 sub { font-size:0.7em; color:var(--accent); }
  .tabs { display:flex; gap:6px; }
  .tab { padding:6px 14px; border-radius:20px; border:1px solid rgba(0,229,255,0.3); background:transparent; color:var(--text); font-family:'Inter',monospace; font-size:0.72rem; cursor:pointer; transition:all 0.2s; }
  .tab.active, .tab:hover { background:var(--accent); color:#000; border-color:var(--accent); }
  .content { flex:1; display:flex; overflow:hidden; position:relative; }
  #structure-panel, #band-panel { width:100%; height:100%; position:absolute; top:0; left:0; }
  #structure-panel { display:flex; }
  #band-panel { display:none; background:var(--bg); }
  #canvas-container { flex:1; position:relative; }
  #three-canvas { width:100% !important; height:100% !important; }
  #legend { position:absolute; bottom:20px; left:20px; background:var(--panel); border:1px solid rgba(0,229,255,0.15); border-radius:12px; padding:12px 16px; backdrop-filter:blur(10px); font-family:'Inter',monospace; font-size:0.72rem; }
  .legend-item { display:flex; align-items:center; gap:8px; margin:4px 0; }
  .dot { width:14px; height:14px; border-radius:50%; flex-shrink:0; }
  #info-box { position:absolute; top:20px; right:20px; background:var(--panel); border:1px solid rgba(0,229,255,0.15); border-radius:12px; padding:14px 16px; backdrop-filter:blur(10px); font-size:0.75rem; line-height:1.7; max-width:210px; }
  .info-label { color:var(--accent); font-family:'Inter',monospace; font-size:0.65rem; }
  #band-panel { flex-direction:column; padding:20px; gap:16px; }
  #band-canvas-wrap { flex:1; position:relative; background:rgba(255,255,255,0.02); border:1px solid rgba(0,229,255,0.12); border-radius:12px; overflow:hidden; }
  #band-canvas { width:100%; height:100%; }
  .band-info { display:flex; gap:16px; flex-shrink:0; flex-wrap:wrap; }
  .band-card { background:var(--panel); border:1px solid rgba(0,229,255,0.15); border-radius:10px; padding:12px 18px; flex:1; min-width:120px; }
  .band-card .val { font-size:1.4rem; font-weight:800; color:var(--accent); }
  .band-card .lbl { font-size:0.68rem; font-family:'Inter',monospace; color:#7a9bb5; }
</style>
</head>
<body>
<div id="app">
  <header>
    <h1>MoSe<sub>2</sub> Viewer</h1>
    <div class="tabs">
      <button class="tab active" onclick="showPanel('structure')">3D Structure</button>
      <button class="tab" onclick="showPanel('band')">Band Structure</button>
    </div>
  </header>
  <div class="content">
    <div id="structure-panel">
      <div id="canvas-container">
        <canvas id="three-canvas"></canvas>
        <div id="legend">
          <div class="legend-item"><div class="dot" style="background:#FF6B35;"></div> Mo (4d orbital)</div>
          <div class="legend-item"><div class="dot" style="background:#A78BFA;"></div> Se (4p orbital)</div>
          <div class="legend-item"><div class="dot" style="background:rgba(255,255,255,0.1);border:1px solid rgba(255,255,255,0.2);"></div> Unit Cell</div>
        </div>
        <div id="info-box">
          <div class="info-label">CRYSTAL INFO</div>
          <div>Space group: P6&#8323;/mmc</div>
          <div>a = 3.288 &#8491;</div>
          <div>c = 12.927 &#8491;</div>
          <div>Mo&#8211;Se bond: 2.54 &#8491;</div>
          <div style="margin-top:6px;" class="info-label">STRUCTURE</div>
          <div>2H polymorph</div>
          <div>Trigonal prismatic</div>
          <div>Mo coordination</div>
        </div>
      </div>
    </div>
    <div id="band-panel">
      <div id="band-canvas-wrap"><canvas id="band-canvas"></canvas></div>
      <div class="band-info">
        <div class="band-card"><div class="val">1.65 eV</div><div class="lbl">DIRECT GAP (MONOLAYER)</div></div>
        <div class="band-card"><div class="val">1.09 eV</div><div class="lbl">INDIRECT GAP (BULK)</div></div>
        <div class="band-card"><div class="val">K &rarr; K</div><div class="lbl">DIRECT GAP LOCATION</div></div>
        <div class="band-card"><div class="val">Mo 4d</div><div class="lbl">VBM CHARACTER</div></div>
      </div>
    </div>
  </div>
</div>
<script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
<script>
// MoSe2 3D viewer
// I made this for my materials science assignment
// it shows the crystal structure and band structure of MoSe2
// MoSe2 parameters: a=3.288A, c=12.927A (larger than MoS2 because Se atoms are bigger than S)
var canvas = document.getElementById('three-canvas');
var container = document.getElementById('canvas-container');
var renderer = new THREE.WebGLRenderer({ canvas, antialias:true, alpha:true });
renderer.setPixelRatio(Math.min(window.devicePixelRatio,2));
renderer.setClearColor(0x050a14,1);
var scene = new THREE.Scene();
var camera = new THREE.PerspectiveCamera(50,1,0.1,100);
camera.position.set(6,5,8);
camera.lookAt(0,0,0);
scene.add(new THREE.AmbientLight(0xffffff,0.3));
const dir1 = new THREE.DirectionalLight(0xFF6B35,1.2); dir1.position.set(5,8,5); scene.add(dir1);
const dir2 = new THREE.DirectionalLight(0xA78BFA,0.6); dir2.position.set(-5,-3,-5); scene.add(dir2);
const pt = new THREE.PointLight(0x00e5ff,0.8,20); pt.position.set(0,4,0); scene.add(pt);
// MoSe2 lattice: a=3.288A, c=12.927A — Se is larger than S so bigger spheres
// I looked up the lattice constants for MoSe2 online
// a = 3.288 angstroms, c = 12.927 angstroms
var SCALE = 0.5; // scale factor to make it fit on screen
var a = 3.288 * SCALE; // lattice constant a in scaled units
var c = 12.927 * SCALE; // lattice constant c in scaled units
const a1=new THREE.Vector3(a,0,0);
const a2=new THREE.Vector3(a*Math.cos(Math.PI/3),a*Math.sin(Math.PI/3),0);
const moMat=new THREE.MeshPhongMaterial({color:0xFF6B35,shininess:120,specular:0xffaa88});
// Se is green (distinct from S yellow)
const seMat=new THREE.MeshPhongMaterial({color:0xA78BFA,shininess:100,specular:0xddccff});
const moGeo=new THREE.SphereGeometry(0.26,24,16);   // Mo radius
const seGeo=new THREE.SphereGeometry(0.22,20,14);   // Se larger than S (0.18)
const bondMat=new THREE.MeshPhongMaterial({color:0x3d6080,transparent:true,opacity:0.6});
// arrays to store where all the atoms go
var moPositions = [];
var sePositions = [];
for(let i=-1;i<=2;i++){
  for(let j=-1;j<=2;j++){
    const base=a1.clone().multiplyScalar(i).add(a2.clone().multiplyScalar(j));
    const moPos=base.clone();
    moPositions.push(moPos.clone());
    // Se sits above and below Mo plane; Mo-Se bond = 2.54A scaled
    const dz=c/4*0.52, dx=a*0.333, dy=a*0.577;
    const seOff=[
      new THREE.Vector3(dx,dy,dz), new THREE.Vector3(-dx*2,0,dz), new THREE.Vector3(dx,-dy,dz),
      new THREE.Vector3(dx,dy,-dz), new THREE.Vector3(-dx*2,0,-dz), new THREE.Vector3(dx,-dy,-dz)
    ];
    seOff.forEach(o=>sePositions.push(moPos.clone().add(o)));
  }
}
moPositions.forEach(p=>{const m=new THREE.Mesh(moGeo,moMat);m.position.copy(p);scene.add(m);});
sePositions.forEach(p=>{const m=new THREE.Mesh(seGeo,seMat);m.position.copy(p);scene.add(m);});
// draws a cylinder between two atoms to show the bond
function addBond(p1, p2){
  const dir=p2.clone().sub(p1),len=dir.length();
  if(len>1.3)return;
  const g=new THREE.CylinderGeometry(0.04,0.04,len,8);
  const m=new THREE.Mesh(g,bondMat);
  m.position.copy(p1.clone().add(p2).multiplyScalar(0.5));
  m.quaternion.setFromUnitVectors(new THREE.Vector3(0,1,0),dir.clone().normalize());
  scene.add(m);
}
moPositions.forEach(mo=>sePositions.forEach(se=>{if(mo.distanceTo(se)<1.3)addBond(mo,se);}));
const cellGeo=new THREE.EdgesGeometry(new THREE.BoxGeometry(a,c*0.58,a));
const cellMat=new THREE.LineBasicMaterial({color:0x00e5ff,transparent:true,opacity:0.2});
const cell=new THREE.LineSegments(cellGeo,cellMat);
cell.position.set(a/2,0,a/2); scene.add(cell);
scene.fog=new THREE.FogExp2(0x050a14,0.08);
const group=new THREE.Group();
scene.children.filter(c=>c.isMesh||c.isLineSegments).forEach(c=>{scene.remove(c);group.add(c);});
scene.add(group);
// variables for mouse dragging (i got help with this part)
var isDragging = false;
var prevX = 0;
var prevY = 0;
var rotX = 0.3;
var rotY = 0;
var pinchDist = null;
// this function makes the canvas the right size
function resize(){
  const w=container.clientWidth,h=container.clientHeight;
  renderer.setSize(w,h,false); camera.aspect=w/h; camera.updateProjectionMatrix();
}
resize();
window.addEventListener('resize',resize);
canvas.addEventListener('mousedown',e=>{isDragging=true;prevX=e.clientX;prevY=e.clientY;});
canvas.addEventListener('mousemove',e=>{if(!isDragging)return;rotY+=(e.clientX-prevX)*0.01;rotX+=(e.clientY-prevY)*0.01;prevX=e.clientX;prevY=e.clientY;});
canvas.addEventListener('mouseup',()=>isDragging=false);
canvas.addEventListener('wheel',e=>{camera.position.multiplyScalar(1+e.deltaY*0.001);});
canvas.addEventListener('touchstart',e=>{
  if(e.touches.length===1){isDragging=true;prevX=e.touches[0].clientX;prevY=e.touches[0].clientY;}
  else if(e.touches.length===2){const dx=e.touches[0].clientX-e.touches[1].clientX,dy=e.touches[0].clientY-e.touches[1].clientY;pinchDist=Math.sqrt(dx*dx+dy*dy);}
},{passive:true});
canvas.addEventListener('touchmove',e=>{
  e.preventDefault();
  if(e.touches.length===1&&isDragging){rotY+=(e.touches[0].clientX-prevX)*0.012;rotX+=(e.touches[0].clientY-prevY)*0.012;prevX=e.touches[0].clientX;prevY=e.touches[0].clientY;}
  else if(e.touches.length===2&&pinchDist){const dx=e.touches[0].clientX-e.touches[1].clientX,dy=e.touches[0].clientY-e.touches[1].clientY,d=Math.sqrt(dx*dx+dy*dy);camera.position.multiplyScalar(pinchDist/d);pinchDist=d;}
},{passive:false});
canvas.addEventListener('touchend',()=>{isDragging=false;pinchDist=null;});
// animation loop - runs every frame
function animate(){
  requestAnimationFrame(animate);
  const r=camera.position.length();
  camera.position.x=r*Math.sin(rotY)*Math.cos(rotX);
  camera.position.y=r*Math.sin(rotX);
  camera.position.z=r*Math.cos(rotY)*Math.cos(rotX);
  camera.lookAt(0,0,0);
  if(!isDragging)rotY+=0.003;
  renderer.render(scene,camera);
}
animate();
function drawBandStructure(){
  const c2=document.getElementById('band-canvas');
  const wrap=document.getElementById('band-canvas-wrap');
  const W=wrap.clientWidth,H=wrap.clientHeight;
  c2.width=W*window.devicePixelRatio; c2.height=H*window.devicePixelRatio;
  c2.style.width=W+'px'; c2.style.height=H+'px';
  const ctx=c2.getContext('2d'); ctx.scale(window.devicePixelRatio,window.devicePixelRatio);
  const pad={l:55,r:25,t:20,b:45},pw=W-pad.l-pad.r,ph=H-pad.t-pad.b;
  ctx.fillStyle='#050a14'; ctx.fillRect(0,0,W,H);
  const kFrac=[0,0.3,0.65,1.0],kLabels=['\u0393','M','K','\u0393'];
  const kX=kFrac.map(f=>pad.l+f*pw);
  const Emin=-4,Emax=4;
  const eScale=e=>pad.t+ph*(1-(e-Emin)/(Emax-Emin));
  ctx.strokeStyle='rgba(0,229,255,0.06)'; ctx.lineWidth=1;
  for(let e=-4;e<=4;e++){const y=eScale(e);ctx.beginPath();ctx.moveTo(pad.l,y);ctx.lineTo(pad.l+pw,y);ctx.stroke();}
  kX.forEach(x=>{ctx.strokeStyle='rgba(255,255,255,0.1)';ctx.lineWidth=1;ctx.setLineDash([4,4]);ctx.beginPath();ctx.moveTo(x,pad.t);ctx.lineTo(x,pad.t+ph);ctx.stroke();ctx.setLineDash([]);});
  ctx.strokeStyle='rgba(255,100,100,0.5)';ctx.lineWidth=1;ctx.setLineDash([6,3]);
  ctx.beginPath();ctx.moveTo(pad.l,eScale(0));ctx.lineTo(pad.l+pw,eScale(0));ctx.stroke();ctx.setLineDash([]);
  ctx.fillStyle='rgba(255,100,100,0.6)';ctx.font='11px monospace';ctx.fillText('E_F',pad.l-30,eScale(0)+4);
  const N=200;
  function interp(kpts,evals,k){
    for(let i=0;i<kpts.length-1;i++){
      if(k>=kpts[i]&&k<=kpts[i+1]){
        const t=(k-kpts[i])/(kpts[i+1]-kpts[i]),tc=(1-Math.cos(t*Math.PI))/2;
        return evals[i]*(1-tc)+evals[i+1]*tc;
      }
    }
    return evals[evals.length-1];
  }
  // MoSe2 band structure — smaller bandgap than MoS2
  // Bulk indirect gap ~1.09 eV (K->Gamma), monolayer direct ~1.65 eV at K
  // VBM is at K (Mo 4d + Se 4p), CBM at K also for monolayer
  const bands=[
    // Valence bands (Se 4p + Mo 4d character) — green tones
    {color:'#A78BFA',knots:[[0,-0.9],[0.3,-1.5],[0.65,0.0],[1.0,-0.9]]},
    {color:'#9061f9',knots:[[0,-1.8],[0.3,-2.5],[0.65,-1.2],[1.0,-1.8]]},
    {color:'#7c3aed',knots:[[0,-2.9],[0.3,-3.1],[0.65,-2.6],[1.0,-2.9]]},
    // Conduction bands (Mo 4d character) — blue tones
    // Bulk CBM slightly lower at K than MoS2
    {color:'#FF6B35',knots:[[0,1.1],[0.3,1.45],[0.65,1.09],[1.0,1.1]]},
    {color:'#fb923c',knots:[[0,2.0],[0.3,1.85],[0.65,2.2],[1.0,2.0]]},
    {color:'#ea580c',knots:[[0,3.1],[0.3,2.8],[0.65,3.0],[1.0,3.1]]}
  ];
  bands.forEach(band=>{
    const kk=band.knots.map(k=>k[0]),ek=band.knots.map(k=>k[1]);
    ctx.beginPath();ctx.strokeStyle=band.color;ctx.lineWidth=2.2;ctx.shadowColor=band.color;ctx.shadowBlur=6;
    for(let i=0;i<N;i++){const k=i/(N-1),e=interp(kk,ek,k),x=pad.l+k*pw,y=eScale(e);i===0?ctx.moveTo(x,y):ctx.lineTo(x,y);}
    ctx.stroke();
  });
  ctx.shadowBlur=0;
  // Bandgap annotation at K
  const kK=kX[2],gapTop=eScale(1.09),gapBot=eScale(0.0);
  ctx.fillStyle='rgba(0,229,255,0.08)';ctx.fillRect(kK-20,gapTop,40,gapBot-gapTop);
  ctx.strokeStyle='#00e5ff';ctx.lineWidth=1.5;ctx.beginPath();ctx.moveTo(kK,gapTop+2);ctx.lineTo(kK,gapBot-2);ctx.stroke();
  ctx.fillStyle='#00e5ff';
  ctx.beginPath();ctx.moveTo(kK,gapTop);ctx.lineTo(kK-4,gapTop+8);ctx.lineTo(kK+4,gapTop+8);ctx.fill();
  ctx.beginPath();ctx.moveTo(kK,gapBot);ctx.lineTo(kK-4,gapBot-8);ctx.lineTo(kK+4,gapBot-8);ctx.fill();
  ctx.fillStyle='#00e5ff';ctx.font='bold 10px monospace';ctx.fillText('1.09eV',kK+6,(gapTop+gapBot)/2+4);
  ctx.strokeStyle='rgba(255,255,255,0.25)';ctx.lineWidth=1;
  ctx.beginPath();ctx.moveTo(pad.l,pad.t);ctx.lineTo(pad.l,pad.t+ph);ctx.stroke();
  ctx.beginPath();ctx.moveTo(pad.l,pad.t+ph);ctx.lineTo(pad.l+pw,pad.t+ph);ctx.stroke();
  ctx.fillStyle='#7a9bb5';ctx.font='11px monospace';ctx.textAlign='right';
  for(let e=-4;e<=4;e+=2)ctx.fillText(e,pad.l-6,eScale(e)+4);
  ctx.fillStyle='#cdd9e5';ctx.font='12px sans-serif';
  ctx.save();ctx.translate(14,pad.t+ph/2);ctx.rotate(-Math.PI/2);ctx.textAlign='center';ctx.fillText('Energy (eV)',0,0);ctx.restore();
  ctx.textAlign='center';
  kLabels.forEach((label,i)=>{ctx.fillStyle='#cdd9e5';ctx.font='bold 13px sans-serif';ctx.fillText(label,kX[i],pad.t+ph+28);});
}
function showPanel(name){
  document.querySelectorAll('.tab').forEach((t,i)=>t.classList.toggle('active',(i===0&&name==='structure')||(i===1&&name==='band')));
  const sp=document.getElementById('structure-panel'),bp=document.getElementById('band-panel');
  if(name==='structure'){sp.style.display='flex';bp.style.display='none';resize();}
  else{sp.style.display='none';bp.style.display='flex';setTimeout(drawBandStructure,50);}
}
window.showPanel=showPanel;
window.addEventListener('resize',()=>{resize();if(document.getElementById('band-panel').style.display!=='none')drawBandStructure();});
</script>
</body>
</html>
'''

display(HTML(f'<div style="height:620px">{html_code}</div>'))

In [ ]:
# Save & download the HTML file to your device
with open('MoSe2_Viewer.html', 'w') as f:
    f.write(html_code)

try:
    from google.colab import files
    files.download('MoSe2_Viewer.html')
    print('Download started! Open in Safari on iPad for full touch support.')
except ImportError:
    print('File saved as MoSe2_Viewer.html')